In [ ]:
dir_save = valid

@contextmanager
def timer(name):
    """
    Time Each Process
    """
    t0 = time.time()
    yield
    print('\n[{}] done in {} Minutes\n'.format(name, round((time.time() - t0)/60,2)))

# Device Features
def id_split(dataframe):
    # https://www.kaggle.com/davidcairuz/feature-engineering-lightgbm-w-gpu
    dataframe['device_name'] = dataframe['DeviceInfo'].str.split('/', expand=True)[0]
    dataframe['device_version'] = dataframe['DeviceInfo'].str.split('/', expand=True)[1]

    dataframe['OS_id_30'] = dataframe['id_30'].str.split(' ', expand=True)[0]
    dataframe['version_id_30'] = dataframe['id_30'].str.split(' ', expand=True)[1]

    dataframe['browser_id_31'] = dataframe['id_31'].str.split(' ', expand=True)[0]
    dataframe['version_id_31'] = dataframe['id_31'].str.split(' ', expand=True)[1]

    dataframe['screen_width'] = dataframe['id_33'].str.split('x', expand=True)[0]
    dataframe['screen_height'] = dataframe['id_33'].str.split('x', expand=True)[1]

    dataframe['id_34'] = dataframe['id_34'].str.split(':', expand=True)[1]
    dataframe['id_23'] = dataframe['id_23'].str.split(':', expand=True)[1]

    dataframe.loc[dataframe['device_name'].str.contains('SM', na=False), 'device_name'] = 'Samsung'
    dataframe.loc[dataframe['device_name'].str.contains('SAMSUNG', na=False), 'device_name'] = 'Samsung'
    dataframe.loc[dataframe['device_name'].str.contains('GT-', na=False), 'device_name'] = 'Samsung'
    dataframe.loc[dataframe['device_name'].str.contains('Moto G', na=False), 'device_name'] = 'Motorola'
    dataframe.loc[dataframe['device_name'].str.contains('Moto', na=False), 'device_name'] = 'Motorola'
    dataframe.loc[dataframe['device_name'].str.contains('moto', na=False), 'device_name'] = 'Motorola'
    dataframe.loc[dataframe['device_name'].str.contains('LG-', na=False), 'device_name'] = 'LG'
    dataframe.loc[dataframe['device_name'].str.contains('rv:', na=False), 'device_name'] = 'RV'
    dataframe.loc[dataframe['device_name'].str.contains('HUAWEI', na=False), 'device_name'] = 'Huawei'
    dataframe.loc[dataframe['device_name'].str.contains('ALE-', na=False), 'device_name'] = 'Huawei'
    dataframe.loc[dataframe['device_name'].str.contains('-L', na=False), 'device_name'] = 'Huawei'
    dataframe.loc[dataframe['device_name'].str.contains('Blade', na=False), 'device_name'] = 'ZTE'
    dataframe.loc[dataframe['device_name'].str.contains('BLADE', na=False), 'device_name'] = 'ZTE'
    dataframe.loc[dataframe['device_name'].str.contains('Linux', na=False), 'device_name'] = 'Linux'
    dataframe.loc[dataframe['device_name'].str.contains('XT', na=False), 'device_name'] = 'Sony'
    dataframe.loc[dataframe['device_name'].str.contains('HTC', na=False), 'device_name'] = 'HTC'
    dataframe.loc[dataframe['device_name'].str.contains('ASUS', na=False), 'device_name'] = 'Asus'

    dataframe.loc[dataframe.device_name.isin(dataframe.device_name.value_counts()[dataframe.device_name.value_counts() < 200].index), 'device_name'] = "Others"
    dataframe['had_id'] = 1
    gc.collect()
    
    return dataframe
    
def fraud_preprocessing(debug = None):
    print("Starting Pre-Processing..")
    with timer("Load Tables"):
        train_transaction = pd.read_csv('../input/train_transaction.csv',
                                        index_col='TransactionID', nrows= debug)
        test_transaction = pd.read_csv('../input/test_transaction.csv',
                                       index_col='TransactionID', nrows= debug)

        train_identity = pd.read_csv('../input/train_identity.csv', index_col='TransactionID')
        test_identity = pd.read_csv('../input/test_identity.csv', index_col='TransactionID')

        
    with timer("Merge Tables"):
        train = train_transaction.merge(train_identity, how='left', left_index=True, right_index=True)
        test = test_transaction.merge(test_identity, how='left', left_index=True, right_index=True)

        print("Train Shape: {} Rows, {} Columns".format(*train.shape))
        print("Test Shape: {} Rows, {} Columns".format(*test.shape))

        y = train[target_var].copy()
        del train_transaction, train_identity, test_transaction, test_identity

        traindex = train.index
        testdex = test.index
        
    with timer("Train/Test Split Feature Engineering"):
        # Credit https://www.kaggle.com/davidcairuz/feature-engineering-lightgbm-w-gpu
        train['gpy_TransactionAmt_to_mean_card1'] = train['TransactionAmt'] / train.groupby(['card1'])['TransactionAmt'].transform('mean')
        train['gpy_TransactionAmt_to_mean_card4'] = train['TransactionAmt'] / train.groupby(['card4'])['TransactionAmt'].transform('mean')
        train['gpy_TransactionAmt_to_std_card1'] = train['TransactionAmt'] / train.groupby(['card1'])['TransactionAmt'].transform('std')
        train['gpy_TransactionAmt_to_std_card4'] = train['TransactionAmt'] / train.groupby(['card4'])['TransactionAmt'].transform('std')

        test['gpy_TransactionAmt_to_mean_card1'] = test['TransactionAmt'] / test.groupby(['card1'])['TransactionAmt'].transform('mean')
        test['gpy_TransactionAmt_to_mean_card4'] = test['TransactionAmt'] / test.groupby(['card4'])['TransactionAmt'].transform('mean')
        test['gpy_TransactionAmt_to_std_card1'] = test['TransactionAmt'] / test.groupby(['card1'])['TransactionAmt'].transform('std')
        test['gpy_TransactionAmt_to_std_card4'] = test['TransactionAmt'] / test.groupby(['card4'])['TransactionAmt'].transform('std')

        train['gpy_id_02_to_mean_card1'] = train['id_02'] / train.groupby(['card1'])['id_02'].transform('mean')
        train['gpy_id_02_to_mean_card4'] = train['id_02'] / train.groupby(['card4'])['id_02'].transform('mean')
        train['gpy_id_02_to_std_card1'] = train['id_02'] / train.groupby(['card1'])['id_02'].transform('std')
        train['gpy_id_02_to_std_card4'] = train['id_02'] / train.groupby(['card4'])['id_02'].transform('std')

        test['gpy_id_02_to_mean_card1'] = test['id_02'] / test.groupby(['card1'])['id_02'].transform('mean')
        test['gpy_id_02_to_mean_card4'] = test['id_02'] / test.groupby(['card4'])['id_02'].transform('mean')
        test['gpy_id_02_to_std_card1'] = test['id_02'] / test.groupby(['card1'])['id_02'].transform('std')
        test['gpy_id_02_to_std_card4'] = test['id_02'] / test.groupby(['card4'])['id_02'].transform('std')

        train['gpy_D15_to_mean_card1'] = train['D15'] / train.groupby(['card1'])['D15'].transform('mean')
        train['gpy_D15_to_mean_card4'] = train['D15'] / train.groupby(['card4'])['D15'].transform('mean')
        train['gpy_D15_to_std_card1'] = train['D15'] / train.groupby(['card1'])['D15'].transform('std')
        train['gpy_D15_to_std_card4'] = train['D15'] / train.groupby(['card4'])['D15'].transform('std')

        test['gpy_D15_to_mean_card1'] = test['D15'] / test.groupby(['card1'])['D15'].transform('mean')
        test['gpy_D15_to_mean_card4'] = test['D15'] / test.groupby(['card4'])['D15'].transform('mean')
        test['gpy_D15_to_std_card1'] = test['D15'] / test.groupby(['card1'])['D15'].transform('std')
        test['gpy_D15_to_std_card4'] = test['D15'] / test.groupby(['card4'])['D15'].transform('std')

        train['gpy_D15_to_mean_addr1'] = train['D15'] / train.groupby(['addr1'])['D15'].transform('mean')
        train['gpy_D15_to_mean_card4'] = train['D15'] / train.groupby(['card4'])['D15'].transform('mean')
        train['gpy_D15_to_std_addr1'] = train['D15'] / train.groupby(['addr1'])['D15'].transform('std')
        train['gpy_D15_to_std_card4'] = train['D15'] / train.groupby(['card4'])['D15'].transform('std')

        test['gpy_D15_to_mean_addr1'] = test['D15'] / test.groupby(['addr1'])['D15'].transform('mean')
        test['gpy_D15_to_mean_card4'] = test['D15'] / test.groupby(['card4'])['D15'].transform('mean')
        test['gpy_D15_to_std_addr1'] = test['D15'] / test.groupby(['addr1'])['D15'].transform('std')
        test['gpy_D15_to_std_card4'] = test['D15'] / test.groupby(['card4'])['D15'].transform('std')
        
    cols_save = [col for col in train.columns if col.count('gpy_')]
    save_feature(train[cols_save], 'ker', dir_save, is_train=True, auto_type=True, list_ignore=COLUMNS_IGNORE)
    save_feature(test[cols_save],  'ker', dir_save, is_train=False, auto_type=True, list_ignore=COLUMNS_IGNORE)